# Skenariolab data Engiiner homework project

This task was pretty fun and actually more timeconsuming that I initially thought. I tried to utilize postgis for extra points, but I faced issues with builging postgis extension for postgres and decided to move on only with Python, which worked out in the end.

Let's start withy importing necessary libraries

In [1]:
import json, requests  #to query data and work with jsons
import geojson # to dump data to geojson file
import xml.etree.ElementTree as ET #to parse cadastral maps data
from shapely.geometry import Polygon, Point  #to work with polygons (areas) and points (buildings)
from geojson import Feature, FeatureCollection #to output work to geojson
from OSMPythonTools.overpass import Overpass #to query for childcare around the selected building with apartment on sale
import haversine as hs #to convert degrees to distances
from haversine import Unit #to convert degrees to distances in meters
#And two universal helpers - pandas and numpy
import pandas as pd
import numpy as np

## 1 Kiinteistomaailma data

### 1.1 Fetching Kiinteistomaailma data and select an apartment on sale
Naturally, I decided to select an apartment in my own neighborhood - Kannelmaki. Luckily, one of the apartments in our asuntoyhtio is on sale.

In [2]:
address = {'street_address': 'Kanneltie 6', 'city': 'Helsinki'}
KM_URL= "https://vilpas.kiinteistomaailma.fi/export/km/listings/baseline.json"
#Fetch KM listings:
listings = requests.get(KM_URL)
listings_text = listings.text
listings_data = json.loads(listings_text)
#The nature of the task is such that I checked beforehand that there is at least one apartment available for sale.
#Therefore I know that listings_data will have elements in it and I can juct pick 1st element
apartment = list(filter(lambda a_listing: (a_listing['address'] == address['street_address']) & 
                        (a_listing['city']==address['city']), listings_data))[0]

In [3]:
# Let's check that found apartment is indeed from Kanneltie 6, Helsinki

print('Correct apartment is found: ', apartment['address']==address['street_address'] and apartment['city']==address['city'])

Correct apartment is found:  True


In [4]:
#getting apartment coordinates - will need to calculate distance to a kindergarden
apartment_lat = apartment['latitude']
apartment_lon = apartment['longitude']

In [5]:
daycares_KM = apartment['dayCare']
print(daycares_KM)

[{'name': 'Päiväkoti  Kannelmäki', 'distance': 0.15}, {'name': 'Päiväkoti Vanhainen', 'distance': 0.35}]


There are at least these two daycares nearby (I can confirm it as well).

### 1.2 Query OSM for daycares nearby
The Overpass API is a read-only API for OSM map data. 
There are many Python wrappers for it in existtence. I have decided to use [OSMPythonTools](https://wiki.openstreetmap.org/wiki/OSMPythonTools) - it is referenced at the OSM wiki and looks straightforward to use.

In [6]:
overpass = Overpass()
# Construct query for Overpass
# I will be searching for all childcare amenities in the radius of twice larger than mentioned in the KM listing.
# That would be awful though if KM lists information with such large errors.

#define radius twice larger than the most far listed daycare:
radius = str(max([a_daycare['distance'] for a_daycare in daycares_KM])*1000*2) 

# OSM wiki mentions three possible tags for a childcare amenity:
childcare_tags = ['kindergarten','nursery','childcare']

# Query OSM for every possible tag
daycares_OSM = [] #will store here all found childcare amenities
for a_tag in childcare_tags:
    a_tag_query = ''.join(['node(around:',radius,'0, ',
                           str(apartment_lat),', ', 
                           str(apartment_lon),')["amenity"="', a_tag, '"];', 
                           'out body;'])
    print("constructed OSm query: ", a_tag_query)
    osm_output = overpass.query(a_tag_query).toJSON()["elements"]
    try: # try if there are any childcares found
        for an_output in osm_output:
            print(an_output)
            daycares_OSM.append({'tags': {
                        "name": an_output['tags']['name'], 
                        "latitude":  an_output['lat'], 
                        "longitude": an_output['lon']}})
    except: #if not found, proceed to the next tag
        continue


constructed OSm query:  node(around:700.00, 60.2410462, 24.8809991)["amenity"="kindergarten"];out body;
{'type': 'node', 'id': 4386743153, 'lat': 60.2387728, 'lon': 24.8840843, 'tags': {'addr:city': 'Helsinki', 'addr:housenumber': '3', 'addr:postcode': '00420', 'addr:street': 'Hanuripolku', 'amenity': 'kindergarten', 'name': 'Suomalais-ranskalainen päiväkoti Les Galopins', 'phone': '09-440407', 'website': 'http://www.lesgalopinspk.com/'}}
{'type': 'node', 'id': 4386751391, 'lat': 60.2398472, 'lon': 24.8796532, 'tags': {'addr:city': 'Helsinki', 'addr:housenumber': '5', 'addr:postcode': '00420', 'addr:street': 'Kaustisenpolku', 'amenity': 'kindergarten', 'name': 'Päiväkoti Kannel'}}
{'type': 'node', 'id': 4386792543, 'lat': 60.2427708, 'lon': 24.8771044, 'tags': {'addr:city': 'Helsinki', 'addr:housenumber': '10', 'addr:postcode': '00420', 'addr:street': 'Soittajantie', 'amenity': 'kindergarten', 'email': 'pk.vanhainen@hel.fi', 'fax': '+358 9 310 41608', 'name': 'Päiväkoti Vanhainen', 'ph

In [7]:
# Print all found childcares
daycares_OSM

[{'tags': {'name': 'Suomalais-ranskalainen päiväkoti Les Galopins',
   'latitude': 60.2387728,
   'longitude': 24.8840843}},
 {'tags': {'name': 'Päiväkoti Kannel',
   'latitude': 60.2398472,
   'longitude': 24.8796532}},
 {'tags': {'name': 'Päiväkoti Vanhainen',
   'latitude': 60.2427708,
   'longitude': 24.8771044}}]

### 1.3 Check if listed distance to the childcare amenities is correct

In [8]:
# Now let's compare KM listed childcares versus OSM found childcares. 
#And if there is a match - calculate distance to the apartment building
distance_OSM=[]
for a_daycare_OSM in daycares_OSM:
    try:
        if a_daycare_OSM['tags']['name'] in [ele['name'] for ele in daycares_KM]: #check for the match
            print('Daycare found: ', a_daycare_OSM['tags']['name'])
            distance_OSM.append({
                'name': a_daycare_OSM['tags']['name'], 
                'distance': 
                hs.haversine((apartment_lat, apartment_lon),
                             (a_daycare_OSM['tags']['latitude'],
                              a_daycare_OSM['tags']['longitude']),
                             unit = Unit.METERS)})
            
    except:
        continue

Daycare found:  Päiväkoti Vanhainen


In [9]:
distance_OSM

[{'name': 'Päiväkoti Vanhainen', 'distance': 288.05942507370287}]

It is worth to mention that OSM data depends on users and their input and can be incomplete or contain errors. In this case, OSM data does not contain information on the closest kindergarten 'Kannelmaki". KM also didn't list paivakoti Kannel which is pretty close as well. But I got one match and will analyze it.

According to OSM, distance to Päiväkoti Vanhainen is 288 meters which is less than 350m as per KM. However, Kanneltie 6 in fact has several buildings that belong to the same street number 6. For more precise distance calculations I would need to know which building exactly apartment belongs to and query for the building with correct entrance number. 

## 2 Cadastral maps

### 2.1 Read maps
I have tried my best to read data with geopandas that should support reading gml files, but there was a constant issue with fiona package and very limited wisdom on it online. I have turned to the more generic XML parsing instead.
The whole following code can be expanded to read a list of data files from a given folder location(s). For this I would need to look more into naming convention of the those maps so that it would be possible know which file is cadastral index map and which is topographic database. 

In [10]:
# Read both files and fetch root of the tree
cadastral_map_root = ET.parse('L4134A.xml').getroot()
topography_map_root = ET.parse('L4134L.xml').getroot()

### 2.2. Topographic Database map data - store buildings' coordinates

I have decided to treat buildings as points - I will be checking if a certain building belong to a cadastral area polygon (since in the topography map buildings are not attributed to any area)

In [11]:
# Search for all buildings ('rakennukset') on the map
buildings = topography_map_root.findall('{http://xml.nls.fi/XML/Namespace/' \
                                        'Maastotietojarjestelma/SiirtotiedostonMalli/2011-02}rakennukset/')

# Start collecting information on coordinates of buildings
buildings_latitudes=[]
buildings_longitudes=[]
for a_building in buildings:
    point = a_building.find('{http://xml.nls.fi/XML/Namespace/Maastotietojarjestelma/SiirtotiedostonMalli/2011-02}sijainti/' \
                            '{http://xml.nls.fi/XML/Namespace/Maastotietojarjestelma/SiirtotiedostonMalli/2011-02}Piste/' \
                            '{http://www.opengis.net/gml}pos') \
                            .text.split()[:-1]
    point = [float(a_coordinate) for a_coordinate in point] # convert to float
    buildings_latitudes.append(point[0])
    buildings_longitudes.append(point[1])
    
buildings_coordinates_df = pd.DataFrame(list(zip(buildings_latitudes,buildings_longitudes)),
                                        columns=['latitudes','longitudes']) #store coordinates as a dataframe
buildings_coordinates_df.head()

,latitudes,longitudes
0,380422.735,6689726.920
1,380340.144,6689749.616
2,380296.102,6689745.508
3,380401.928,6689784.208
4,380391.483,6689800.124


### 2.3 Cadastral areas map data

I did not figure out fast enough how to avoid brute-forcing tags during search through data - this is something that could be improved overall.

In [12]:
# Find and store "kiinteistotunnus"
cadastral_areas = cadastral_map_root.findall('{http://xml.nls.fi/XML/Namespace/Kiinteistorekisterikartta/2017-01}Palstat/' \
                                             '{http://xml.nls.fi/XML/Namespace/Kiinteistorekisterikartta/2017-01}Palsta/' \
                                             '{http://xml.nls.fi/XML/Namespace/Kiinteistorekisterikartta/2017-01}kiinteistotunnus') #note no backslash here

cadastral_numbers = [area.text for area in cadastral_areas]
print("kiinteistotunnus values stored as list of strings: ", cadastral_numbers[:5])
# Fina all information on position - "sijainti"
cadastral_positions = cadastral_map_root.findall('{http://xml.nls.fi/XML/Namespace/Kiinteistorekisterikartta/2017-01}Palstat/' \
                                                 '{http://xml.nls.fi/XML/Namespace/Kiinteistorekisterikartta/2017-01}Palsta/' \
                                                 '{http://xml.nls.fi/XML/Namespace/Kiinteistorekisterikartta/2017-01}sijainti')


kiinteistotunnus values stored as list of strings:  ['09143100010659', '09142100010003', '09140400010002', '09103499010000', '09289500020001']


I will be going over every cadastral area one by one and search for the buildings that belong to it.

In [13]:
cadastral_polygons = []
count = 0
count_buildings = 0
geojsonFeatures=[] # I will store here features

for an_area in cadastral_positions:
    
    #1. get coordinates of the exterior line of cadastral area polygon
    exterior=an_area.find('{http://www.opengis.net/gml}Surface/' \
                          '{http://www.opengis.net/gml}patches/' \
                          '{http://www.opengis.net/gml}PolygonPatch/' \
                          '{http://www.opengis.net/gml}exterior/' \
                          '{http://www.opengis.net/gml}LinearRing/' \
                          '{http://www.opengis.net/gml}posList')
    exterior = exterior.text.split()
    exterior = [float(ele) for ele in exterior] #convert to floats
    iterator = iter(exterior)
    exterior_coordinates = list(zip(iterator, iterator)) #convert exterior ring coordinates to the list of tuple - will is needed for shapely Polygon objects

    #2. search for any possible holes in the cadastral area polygon
    interiors = an_area.findall('{http://www.opengis.net/gml}Surface/{http://www.opengis.net/gml}patches/' \
                                '{http://www.opengis.net/gml}PolygonPatch/' \
                                '{http://www.opengis.net/gml}interior/' \
                                '{http://www.opengis.net/gml}LinearRing/' \
                                '{http://www.opengis.net/gml}posList')
    
    if len(interiors) > 0: #if there are holes in the polygon:
        holes = [] #we need to store all coordinates for holes
        for interior in interiors:
            interior = [float(ele) for ele in interior.text.split()] 
            iterator = iter(interior)
            interior_coordinates = list(zip(iterator,iterator)) #this will be a list of tuples as shapely Polygon expects
            holes.append(interior_coordinates)
        cadastral_polygon=Polygon(exterior_coordinates, holes) #constructing polygon with holes
    else:
        cadastral_polygon=Polygon(exterior_coordinates)  #constructiong poluygon without holes
    cadastral_polygons.append(cadastral_polygon) #append a polygon to the list of polygons
    
    area = cadastral_polygon.area/10000 #area in ha - divided by 100x100m 
    
    # Search for buildings that belong to every area
    # Checking building one by one would be very very slow. 
    # To speed up the search I will only consider buildings that can be atrributed to a rectangle 
    # where any given polygon resides.
    
    # 1. find rectangle where polygon resides
    (minx, miny, maxx, maxy) = cadastral_polygon.bounds 
    # 2. find all buildings in a given rectangle - store as dataframe
    is_in_rectangle=buildings_coordinates_df.loc[(buildings_coordinates_df["latitudes"] > minx) & 
                                                 (buildings_coordinates_df["latitudes"] < maxx) & 
                                                 (buildings_coordinates_df["longitudes"] > miny) &  
                                                 (buildings_coordinates_df["longitudes"] < maxy)]
    # 3. check if buildings actually lay within a polygon
    gids = []
    for idx, row in is_in_rectangle.iterrows():
        belongs = cadastral_polygon.contains(Point(row["latitudes"], row["longitudes"]))
        if belongs:
            count_buildings+=1
            gids.append(buildings[idx].attrib["gid"])
 
    # Append to the list of geojson features - one feature per cadastral zone. Geometry as compulsory field is left empty. Rest of the information is stored in properties.
    geojsonFeatures.append(Feature(geometry=[], properties={'gid': gids, 
                                                            "kiinteistotunnus": cadastral_numbers[count], 
                                                            "area": area}))
    count += 1
    
print("Buildings found and atrributed:", count_buildings)

Buildings found and atrributed: 11054


### 2.4 Data on apartment and geojson - how to store?

I had two ideas on how to store Kiinteistomaailma data on apartment, especially thinking if this task would need to be scaled to all apartments on sale:
1. Attach to a certain gid (basically, certain building) apartment info.
2. Store as separate feature.

With this in mind I tried to determine to which gid I could atribute the apartment. I have converted gid coordinates in meters to degrees:

In [14]:
import pyproj
from pyproj import Transformer
transformer = Transformer.from_crs("EPSG:4326", "EPSG:3067") #transfrom from degrees to meters
apartment_lat_meters, apartment_lon_meters = transformer.transform(apartment_lat, apartment_lon)
print("Converted coordinates: ", apartment_lat_meters, apartment_lon_meters)

Converted coordinates:  382681.464419071 6680140.096565886


In [15]:
buildings_coordinates_df[(round(buildings_coordinates_df['longitudes'])==round(apartment_lon_meters))]# 

,latitudes,longitudes
30290,390982.128,6680139.842
32196,386946.128,6680140.426
34303,388640.298,6680139.981
34310,388598.559,6680140.248
40126,383708.901,6680139.679


There is no match between converted coordinates in meters and cadastral map data - I tried to check with different rounding precision and also check if there is actually something reasonably nearby, but I don't think I can make any conclusion based on my result. Perhaps my assumption on projection for KM coordinates is wrong, but I couldn't find more information on the source of the data on coodinrates they provide.

Considering this, KM data will be stored as separate feature since I couldn't attribute it to a particular building. I overall suspect some issue with coordinate conversion here - adding this info to a cadastrial area feature can result in mistake as well.

In [16]:
#make a Feature for KM apartment and append to Features list
KM_feature = Feature(geometry=Point((apartment_lat, apartment_lon)), properties=apartment) 
geojsonFeatures.append(KM_feature)

In [17]:
geojsonCollection = FeatureCollection(geojsonFeatures)

In [18]:
geojsonCollection[0]

{"geometry": null, "properties": {"area": 171.72254196960557, "gid": ["960703220", "934878258", "414341982", "935164269", "414343050", "1974234564", "1930950695", "1930950685", "414364887"], "kiinteistotunnus": "09143100010659"}, "type": "Feature"}

In [19]:
with open('cadastral_objects_info.geojson', 'w') as f:
    geojson.dump(geojsonCollection, f)

That's it folks!